In [2]:
# PS: Deliverable: An agent with at least two HITL checkpoints and a feedback collection mechanism.

# ===============================
# INSTALL
# ===============================
# !pip install -U langgraph langchain langchain-openai

import json
from typing import TypedDict, Optional, List, Dict
from langgraph.graph import StateGraph, END
from langgraph.types import interrupt
from langchain_openai import ChatOpenAI

# ===============================
# FEEDBACK STORE
# ===============================
DRAFT_FEEDBACK: List[Dict] = []

# ===============================
# STATE
# ===============================
class SupportState(TypedDict):
    customerMessage: str

    autoReplyAllowed: bool
    confidence: float

    draftReply: Optional[str]
    approvedReply: Optional[str]

    approved: bool

    feedbackRating: Optional[int]
    feedbackComment: Optional[str]

# ===============================
# LLM
# ===============================
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ===============================
# NODES
# ===============================

def assess_auto_reply(state: SupportState) -> SupportState:
    prompt = f"""
    You are a support operations assistant.

    Decide if this message can be safely auto-replied.
    Return STRICT JSON:

    {{
    "autoReplyAllowed": true|false,
    "confidence": number between 0 and 1
    }}

    Message:
    "{state['customerMessage']}"
    """
    resp = llm.invoke([{"role": "user", "content": prompt}])
    data = json.loads(resp.content)

    state["autoReplyAllowed"] = data["autoReplyAllowed"]
    state["confidence"] = float(data["confidence"])
    return state


# 🔴 HITL #1 — LOW CONFIDENCE AUTO-REPLY
def auto_reply_confidence_gate(state: SupportState) -> SupportState:
    if state["confidence"] < 0.75:
        interrupt({
            "type": "auto_reply_uncertain",
            "message": "Low confidence auto-reply decision",
            "confidence": state["confidence"],
            "messagePreview": state["customerMessage"]
        })
    return state


def draft_reply(state: SupportState) -> SupportState:
    if not state["autoReplyAllowed"]:
        state["draftReply"] = "Escalating this issue to a human support agent."
        return state

    resp = llm.invoke([
        HumanMessage(content=f"""
Draft a professional support reply:

Customer message:
"{state['customerMessage']}"
""")
    ])
    state["draftReply"] = resp.content
    return state


# 🔴 HITL #2 — HUMAN APPROVES OR EDITS DRAFT
def approval_gate(state: SupportState) -> SupportState:
    if not state["approved"]:
        interrupt({
            "type": "approve_reply",
            "draft": state["draftReply"]
        })
    return state


def send_reply(state: SupportState) -> SupportState:
    state["approvedReply"] = state["approvedReply"] or state["draftReply"]
    print("📤 Sending reply:\n", state["approvedReply"])
    return state


# 🟡 FEEDBACK
def feedback_gate(state: SupportState) -> SupportState:
    if state["feedbackRating"] is None:
        interrupt({
            "type": "draft_feedback",
            "draft": state["draftReply"]
        })
    return state


def store_feedback(state: SupportState) -> SupportState:
    DRAFT_FEEDBACK.append({
        "message": state["customerMessage"],
        "draft": state["draftReply"],
        "rating": state["feedbackRating"],
        "comment": state["feedbackComment"]
    })
    return state

# ===============================
# GRAPH
# ===============================
builder = StateGraph(SupportState)

builder.add_node("assess", assess_auto_reply)
builder.add_node("confidence_gate", auto_reply_confidence_gate)
builder.add_node("draft", draft_reply)
builder.add_node("approval_gate", approval_gate)
builder.add_node("send", send_reply)
builder.add_node("feedback_gate", feedback_gate)
builder.add_node("store_feedback", store_feedback)

builder.set_entry_point("assess")

builder.add_edge("assess", "confidence_gate")
builder.add_edge("confidence_gate", "draft")
builder.add_edge("draft", "approval_gate")
builder.add_edge("approval_gate", "send")
builder.add_edge("send", "feedback_gate")
builder.add_edge("feedback_gate", "store_feedback")
builder.add_edge("store_feedback", END)

graph = builder.compile()

# ===============================
# RUN
# ===============================
state = {
    "customerMessage": "I was charged twice for my subscription last month.",
    "autoReplyAllowed": False,
    "confidence": 0.0,
    "draftReply": None,
    "approvedReply": None,
    "approved": False,
    "feedbackRating": None,
    "feedbackComment": None
}

# 1️⃣ Run → may pause at confidence
state = graph.invoke(state)

# Human decides auto-reply is OK
state["autoReplyAllowed"] = True

# 2️⃣ Resume → pause at approval
state = graph.invoke(state)

# Human edits & approves
state["approved"] = True
state["approvedReply"] = "Thanks for reaching out. We've refunded the duplicate charge."

# 3️⃣ Resume → pause at feedback
state = graph.invoke(state)

# Feedback
state["feedbackRating"] = 5
state["feedbackComment"] = "Draft was accurate and saved time."

final_state = graph.invoke(state)

print("✅ Workflow complete")
print("📊 Feedback:", DRAFT_FEEDBACK)


📤 Sending reply:
 Thanks for reaching out. We've refunded the duplicate charge.
📤 Sending reply:
 Thanks for reaching out. We've refunded the duplicate charge.
✅ Workflow complete
📊 Feedback: [{'message': 'I was charged twice for my subscription last month.', 'draft': 'Escalating this issue to a human support agent.', 'rating': 5, 'comment': 'Draft was accurate and saved time.'}]
